In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer

from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.base import clone

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


In [18]:
def evalua_metodo(modelo, X_train, y_train, X_test, y_test, mostrar=False):
    
    # escalamos en la funcion porque a veces no funciona, por las dudas
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)  # escalamos los datos de entrenamiento
    X_test = scaler.fit_transform(X_test)  # escalamos los datos de prueba
    
    modelo.fit(X_train, y_train) 
    
    # Predecimos con los datos de prueba
    y_pred = modelo.predict(X_test)
    y_proba = modelo.predict_proba(X_test)[:, 1]  

    # calcula las metricas que vamos a reprotar
    matriz_confusion = confusion_matrix(y_test, y_pred) 
    fpr, tpr, _ = roc_curve(y_test, y_proba) 
    
    # estas son las que vamos a usar para comparar:
    score_auc = auc(fpr, tpr)  
    accuracy = accuracy_score(y_test, y_pred)  
    mse = mean_squared_error(y_test, y_pred)  
    
    # si mostrar es true, muestra la curva ROC
    if mostrar:
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f"AUC = {score_auc:.2f}")  # grafica la curva ROC
        plt.plot([0, 1], [0, 1], 'r--')  # linea de no habilidad
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.title('ROC Curve')
        plt.legend()
        plt.show()

    # devuelve las metricas en un diccionario
    return { 'Matriz de Confusión': matriz_confusion, 'AUC': score_auc,
        'Accuracy': accuracy, 'MSE': mse
    }

def cross_validation(modelo, k, X, y):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)  # configuracion del k-fold
    resultados = {
        'Accuracy': [],
        'AUC': [],
        'MSE': []
    }

    for train_index, test_index in kfold.split(X):  # divide los datos en k folds
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        resultado = evalua_metodo(modelo, X_train, y_train, X_test, y_test)
        
        # agrega los resultados de cada fold a las listas
        resultados['AUC'].append(resultado['AUC'])
        resultados['Accuracy'].append(resultado['Accuracy'])
        resultados['MSE'].append(resultado['MSE'])

    # calcula el promedio de cv de las medidas
    promedios = {key: np.mean(val) for key, val in resultados.items()}
    
    return promedios

def evalua_config(modelo_base, configs, k, X, y):
    resultados = []
    for config in configs:
        modelo = clone(modelo_base)
        modelo.set_params(**config)
        resultado_cv = cross_validation(modelo, k, X, y)
        resultados.append({
            'configuracion': config,
            'error_promedio': resultado_cv['MSE']
        })
    
    mejor_resultado = min(resultados, key=lambda x: x['error_promedio']) # agarra la config con menor mse
    return mejor_resultado

def evalua_multiples_metodos(X, y):
    # divide datos en entrenamiento, validacion y prueba
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)
    
    # configs para regresion logistica barriendo lambdas
    configuraciones_lr = [{'C': 1/l, 'penalty': 'l2', 'max_iter': 200} for l in np.logspace(-5, 5, 11)]
    
    # Buscamos el mejor lambda
    mejor_config_lr = evalua_config(LogisticRegression(solver='liblinear'), configuraciones_lr, 5, X_train, y_train)
    modelo_lr = LogisticRegression(**mejor_config_lr['configuracion'], solver='liblinear')
    
    # evalua modelos
    resultados = []
    modelos = {
        'Logistic Regression': modelo_lr,
        'LDA': LDA(),
        'KNN (K=3)': KNeighborsClassifier(n_neighbors=3)
    }
    
    for nombre, modelo in modelos.items():
        # evaluacion final con configuracion optima
        metricas = evalua_metodo(modelo, X_train_val, y_train_val, X_test, y_test)
        resultados.append({
            'Modelo': nombre,
            # esto para que sea mas legible el output:
            'Configuración': ", ".join([f"{k}: {v}" for k, v in modelo.get_params().items() if k in ['C', 'penalty', 'n_neighbors']]),
            'AUC': metricas['AUC'],
            'Accuracy': metricas['Accuracy'],
            'MSE': metricas['MSE']
        })
    
    df_resultados = pd.DataFrame(resultados)
    # formatea las columnas numericas
    df_resultados[['AUC', 'Accuracy', 'MSE']] = df_resultados[['AUC', 'Accuracy', 'MSE']].applymap(lambda x: f"{x:.3f}")
    return df_resultados


In [19]:

data = load_breast_cancer()
X = data.data
y = data.target

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instanciar los modelos
modelo_lda = LDA()
modelo_knn = KNeighborsClassifier(n_neighbors=5)
modelo_logistic = LogisticRegression()

# Evaluar LDA
print("KNN:", evalua_metodo(modelo_lda, X_train, y_train, X_test, y_test))
print("KNN CV:", cross_validation(modelo_lda, 5, X, y))

# Evaluar KNN
print("KNN:", evalua_metodo(modelo_knn, X_train, y_train, X_test, y_test))
print("KNN CV:", cross_validation(modelo_knn, 5, X, y))

# Evaluar Logistic Regression
print("Logistic Regression:", evalua_metodo(modelo_logistic, X_train, y_train, X_test, y_test))
print("Logistic Regression CV:", cross_validation(modelo_logistic, 5, X, y))
configuraciones_lr = [{'C': 1/l, 'penalty': 'l2', 'max_iter': 500} for l in np.logspace(-5,5,11)]
mejor_config_lr = evalua_config(LogisticRegression(), configuraciones_lr, 5, X, y)
print("Mejor configuración para Logistic Regression:", mejor_config_lr)


# Uso de la función
df_resultados = evalua_multiples_metodos(X, y)
print(df_resultados)

KNN: {'Matriz de Confusión': array([[ 56,   7],
       [  2, 106]]), 'AUC': 0.9907407407407407, 'Accuracy': 0.9473684210526315, 'MSE': 0.05263157894736842}
KNN CV: {'Accuracy': 0.9472442167365317, 'AUC': 0.9877519236729324, 'MSE': 0.0527557832634684}
KNN: {'Matriz de Confusión': array([[ 59,   4],
       [  1, 107]]), 'AUC': 0.9805261610817166, 'Accuracy': 0.9707602339181286, 'MSE': 0.029239766081871343}
KNN CV: {'Accuracy': 0.9507529886663562, 'AUC': 0.9864858906508879, 'MSE': 0.04924701133364384}
Logistic Regression: {'Matriz de Confusión': array([[ 60,   3],
       [  1, 107]]), 'AUC': 0.9979423868312757, 'Accuracy': 0.9766081871345029, 'MSE': 0.023391812865497075}
Logistic Regression CV: {'Accuracy': 0.9753920198726906, 'AUC': 0.9954850276788912, 'MSE': 0.02460798012730942}
Mejor configuración para Logistic Regression: {'configuracion': {'C': 1.0, 'penalty': 'l2', 'max_iter': 500}, 'error_promedio': 0.02460798012730942}
                Modelo        Configuración    AUC Accuracy   